In [0]:
!pip install --upgrade tables
!pip install eli5

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/dw_matrix/matrix_two/dw_matrix_car'

/content/drive/My Drive/Colab Notebooks/matrix/dw_matrix/matrix_two/dw_matrix_car


##Wczytywanie danych

In [0]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

##Dummy model

In [0]:
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [0]:
feats = ['car_id']
X = df[feats].values
y = df['price_value'].values

model = DummyRegressor()

model.fit(X, y)
y_pred = model.predict(X)

mae(y, y_pred)

39465.934630440985

In [0]:
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [0]:
df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [0]:
df = df[df['price_currency'] == 'PLN']
df.shape

(106290, 155)

##Features

In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if SUFFIX_CAT in feat: continue
  if isinstance(df[feat][0], list): continue
  df[feat + SUFFIX_CAT] = df[feat].factorize()[0]


In [0]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
X = df[cat_feats].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)

-19566.588937368324

In [0]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(X, y)

imp = PermutationImportance(m, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)


Weight,Feature
0.2533 ± 0.0040,param_napęd__CAT
0.2008 ± 0.0047,param_faktura-vat__CAT
0.1943 ± 0.0088,param_stan__CAT
0.1423 ± 0.0086,param_rok-produkcji__CAT
0.0629 ± 0.0047,param_moc__CAT
0.0424 ± 0.0014,feature_kamera-cofania__CAT
0.0412 ± 0.0008,param_skrzynia-biegów__CAT
0.0286 ± 0.0037,param_marka-pojazdu__CAT
0.0191 ± 0.0022,param_pojemność-skokowa__CAT
0.0163 ± 0.0005,feature_bluetooth__CAT
